<a href="https://colab.research.google.com/github/sarunsmenon/llm/blob/main/LLM_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Libraries

In [5]:
! pip install -q deepeval python-dotenv openai langchain-openai langchain langchain_community faiss-cpu  uuid7 langgraph chromadb aiobotocore

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 101.9 MB/s eta 0:00:00


# Load Libraries

In [6]:
from google.colab import userdata
import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import chromadb
import random

In [24]:
from deepeval.synthesizer import Synthesizer
from deepeval import evaluate
from deepeval.metrics import AnswerRelevancyMetric, FaithfulnessMetric
from deepeval.dataset import EvaluationDataset
from deepeval.test_case import LLMTestCase

In [8]:
import requests
import chromadb
from bs4 import BeautifulSoup
import openai
import hashlib
import time
import re

# Load Variables

In [9]:
os.environ['OPENAI_API_KEY'] = userdata.get('open_ai_key')

In [10]:
llm_model = 'gpt-3.5-turbo-1106'
llm = ChatOpenAI(model=llm_model, temperature=0)

In [11]:
# Start crawling from the initial URL
ignore_lst = []
max_pg_lmt = 5000
fldr = '/content/drive/MyDrive/GenAI - Hack/mhdb'

# Create Chroma Db

In [ ]:


def download_and_store_urls_with_openai(urls, openai_api_key, collection_name="web_content"):
    """
    Download content from URLs and store in ChromaDB using OpenAI embeddings

    Args:
        urls: List of URLs to download
        openai_api_key: Your OpenAI API key
        collection_name: Name of the ChromaDB collection
    """
    # Set OpenAI API key
    openai.api_key = openai_api_key

    # Initialize ChromaDB
    client = chromadb.Client()

    # Create or get collection - note we're not providing embeddings
    # ChromaDB will use the documents to create embeddings
    collection = client.get_or_create_collection(
        name=collection_name,
        metadata={"description": "Web content with OpenAI embeddings"}
    )

    # Process each URL
    for url in urls:
        try:
            print(f"Processing {url}")

            # Download content with timeout and headers
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
            }
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()

            # Parse HTML
            soup = BeautifulSoup(response.text, 'html.parser')

            # Extract text content
            for script in soup(["script", "style", "header", "footer", "nav"]):
                script.extract()

            # Get text and clean it
            text = soup.get_text()
            text = re.sub(r'\s+', ' ', text).strip()

            # Break into chunks
            chunks = split_into_chunks(text, chunk_size=1000)

            # Process each chunk
            for i, chunk in enumerate(chunks):
                if not chunk.strip():
                    continue

                # Create a unique ID
                doc_id = hashlib.md5(f"{url}_{i}".encode()).hexdigest()

                # Generate OpenAI embedding
                embedding_response = openai.embeddings.create(
                    model="text-embedding-ada-002",
                    input=chunk
                )
                embedding = embedding_response.data[0].embedding

                # Store in ChromaDB
                collection.add(
                    ids=[doc_id],
                    documents=[chunk],
                    embeddings=[embedding],
                    metadatas=[{
                        "url": url,
                        "chunk_number": i,
                        "total_chunks": len(chunks),
                        "timestamp": time.time()
                    }]
                )

            print(f"Successfully stored {len(chunks)} chunks from {url}")

        except Exception as e:
            print(f"Error processing {url}: {str(e)}")
            continue

        # Be nice to servers
        time.sleep(1)

    print(f"Completed processing {len(urls)} URLs")
    return collection

def split_into_chunks(text, chunk_size=1000, overlap=100):
    """Split text into overlapping chunks of approximately chunk_size characters"""
    chunks = []
    if len(text) <= chunk_size:
        chunks.append(text)
    else:
        start = 0
        while start < len(text):
            end = start + chunk_size
            if end >= len(text):
                chunks.append(text[start:])
            else:
                # Try to find a good breaking point
                break_point = text.rfind(". ", start, end)
                if break_point != -1:
                    chunks.append(text[start:break_point+1])
                    start = break_point + 1 - overlap
                else:
                    chunks.append(text[start:end])
                    start = end - overlap
            start = max(start, 0)
    return chunks

# Example usage
urls = [
    "https://example.com",
    "https://en.wikipedia.org/wiki/Vector_database"
]

collection = download_and_store_urls_with_openai(
    urls,
    openai_api_key="your-openai-api-key"
)

# Load Contexts

## Load Chroma Collection

In [12]:
chroma_client = chromadb.PersistentClient(path=fldr)
collection = chroma_client.get_or_create_collection(name="semantic")

## Load Contexts

In [13]:
all_contexts = collection.get(include=["documents"])["documents"]

In [14]:
sample_context = random.sample(all_contexts, 10)

In [15]:
len(sample_context)

10

In [16]:
sample_context_lst = []
for context in sample_context:
  sample_context_lst.append([context])

# Synthesizer

In [27]:
synthesizer = Synthesizer(model=llm_model)
test_cases = synthesizer.generate_goldens_from_contexts(
    # Provide a list of context for synthetic data generation
    contexts=sample_context_lst,
    max_goldens_per_context = 2
)

✨ Generating up to 20 goldens using DeepEval (using gpt-3.5-turbo-1106, method=default): 100%|██████████| 20/20 [00:13<00:00,  1.49it/s]


In [28]:
dataframe = synthesizer.to_pandas()
print(dataframe)

                                                input actual_output  \
0   What is the method for reaching out to your tr...          None   
1   What support is available for managing increas...          None   
2   Provide a summary of the findings that highlig...          None   
3   How can updates for the Services Directory be ...          None   
4   Imagine a situation where you urgently needed ...          None   
5   What self-administered tool can be used to mon...          None   
6   Examine the content of the Trauma Informed Lea...          None   
7   How many mental health plans does the State ha...          None   
8   Which publication contains a study exploring e...          None   
9   How does stigma and abuse contribute to higher...          None   
10  What are the obstacles causing the fragmentati...          None   
11  Does the Department of Health oversee the mana...          None   
12  What is the objective of the Department of Hea...          None   
13  Wh

# Evaluation

In [19]:
synthesizer.synthetic_goldens[0].input

'Where can I submit updates for the Services Directory Aged Services, including metropolitan and rural maps, and contact details for general enquiries?'

In [20]:
synthesizer.synthetic_goldens[0]

Golden(input='Where can I submit updates for the Services Directory Aged Services, including metropolitan and rural maps, and contact details for general enquiries?', actual_output=None, expected_output='You can submit updates for the Services Directory Aged Services, including metropolitan and rural maps, by clicking on the provided link. For general enquiries, you can contact the Department of Health at 61 3 90960000.', context=['Click \r\n                    here to send us your update to the Services Directory Aged Services - Metropolitan Maps Central & Outer\r\n              East Dandenong Inner & North\r\n              West Inner\r\n              South East Inner\r\n              Urban East & Northcote Mid & South\r\n              West Middle South Northern & North\r\n              East Peninsula Aged Services -\r\n                Rural Maps Barwon Gippsland Glenelg (South Western) Goulburn & Southern Grampians Loddon Campaspe / Southern Mallee North East Hume Northern\r\n       

In [ ]:
type(synthesizer.synthetic_goldens[0])

deepeval.dataset.golden.Golden

In [34]:

# Prepare evaluation metrics
metrics_lst = [
    AnswerRelevancyMetric(threshold=0.7),
    # FaithfulnessMetric(threshold=0.7)
]

# Create test cases from goldens
test_case = []
for golden in synthesizer.synthetic_goldens:
    res = llm.invoke(golden.input).content
    test_case = LLMTestCase(input=golden.input, actual_output=res, expected_output=golden.expected_output)
    print(test_case)
    test_cases.append(test_case)
    evaluate(test_cases=[test_case], metrics=metrics_lst)



# Evaluate end-to-end
# evaluate(test_cases=test_cases, metrics=[AnswerRelevancyMetric()])



LLMTestCase(input='What is the method for reaching out to your treating team for help?', actual_output="1. Contact your primary care physician or therapist: If you have a primary care physician or therapist, reach out to them first. They can provide guidance and support or refer you to a specialist if needed.\n\n2. Use the contact information provided by your treating team: If you have been given contact information for your treating team, such as a phone number or email address, use that to reach out for help.\n\n3. Schedule an appointment: If you have a regular appointment scheduled with your treating team, make sure to attend it and discuss your concerns during the appointment.\n\n4. Utilize telehealth options: Many healthcare providers offer telehealth services, allowing you to connect with your treating team remotely through video calls or phone consultations.\n\n5. Seek emergency help if needed: If you are in crisis or need immediate assistance, do not hesitate to call emergency 

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)...

Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 00:09,  9.83s/test case]



Metrics Summary

  - ✅ Answer Relevancy (score: 0.8235294117647058, threshold: 0.7, strict: False, evaluation model: gpt-4o, reason: The score is 0.82 because the response effectively addresses the method for reaching out to the treating team, but includes some irrelevant statements about emotional encouragement and the role of the treating team, which do not directly answer the question., error: None)

For test case:

  - input: What is the method for reaching out to your treating team for help?
  - actual output: 1. Contact your primary care physician or therapist: If you have a primary care physician or therapist, reach out to them first. They can provide guidance and support or refer you to a specialist if needed.

2. Use the contact information provided by your treating team: If you have been given contact information for your treating team, such as a phone number or email address, use that to reach out for help.

3. Schedule an appointment: If you have a regular appointment sch

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

...

LLMTestCase(input='Imagine a year where VICSERV facilitated consultations, represented sector issues, and provided workplace consultations to enhance mental health services.', actual_output="In this hypothetical year, VICSERV, as a leading mental health advocacy organization, played a crucial role in facilitating consultations with various stakeholders in the mental health sector. These consultations aimed to gather input and feedback from individuals with lived experience, mental health professionals, and community organizations to better understand the current challenges and needs within the mental health system.\n\nAdditionally, VICSERV actively represented sector issues at various forums, including government meetings, policy discussions, and public events. By advocating for improved mental health services and resources, VICSERV worked to ensure that the voices of those affected by mental illness were heard and considered in decision-making processes.\n\nFurthermore, VICSERV provid

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)...

Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 00:07,  7.42s/test case]



Metrics Summary

  - ✅ Answer Relevancy (score: 1.0, threshold: 0.7, strict: False, evaluation model: gpt-4o, reason: The score is 1.00 because the output perfectly aligns with the input, addressing all aspects without any irrelevant statements. Great job on maintaining focus and relevance!, error: None)

For test case:

  - input: Imagine a year where VICSERV facilitated consultations, represented sector issues, and provided workplace consultations to enhance mental health services.
  - actual output: In this hypothetical year, VICSERV, as a leading mental health advocacy organization, played a crucial role in facilitating consultations with various stakeholders in the mental health sector. These consultations aimed to gather input and feedback from individuals with lived experience, mental health professionals, and community organizations to better understand the current challenges and needs within the mental health system.

Additionally, VICSERV actively represented sector issues 

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

In [35]:
# Evaluate end-to-end
evaluate(test_cases=test_cases, metrics=[AnswerRelevancyMetric()])


ValueError: You cannot supply a mixture of `LLMTestCase`/`MLLMTestCase`(s) and `ConversationalTestCase`(s) as the list of test cases.

# Testing

In [ ]:
from deepeval.test_case import LLMTestCase
from deepeval.metrics import AnswerRelevancyMetric

answer_relevancy_metric = AnswerRelevancyMetric()
test_case = LLMTestCase(
  input="Who is the current president of the United States of America?",
  actual_output="Joe Bison",
  retrieval_context=["Joe Biden serves as the current president of America."]
)

answer_relevancy_metric.measure(test_case)
print(answer_relevancy_metric.score)

Output()

0.0


In [ ]:

from deepeval.synthesizer import Synthesizer
from deepeval.dataset import EvaluationDataset

synthesizer = Synthesizer()
goldens = synthesizer.generate_goldens_from_docs(document_paths=['/content/Diwali - Wikipedia.pdf'])

dataset = EvaluationDataset(goldens=goldens)

✨ 🧩 ✨ Generating Contexts: 100%|██████████| 9/9 [00:03<00:00,  2.54it/s]


Utilizing 9 out of 50 chunks.

✨ Generating up to 6 goldens using DeepEval (using gpt-4o and text-embedding-3-small, method=docs):  17%|█▋        | 1/6 [00:08<00:42,  8.48s/it]ERROR:root:OpenAI Error: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-VxwfSaIZ7gLnAsbaBUs3WgIR on tokens per min (TPM): Limit 30000, Used 28947, Requested 2940. Please try again in 3.774s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}} Retrying: 1 time(s)...
ERROR:root:OpenAI Error: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-VxwfSaIZ7gLnAsbaBUs3WgIR on tokens per min (TPM): Limit 30000, Used 28683, Requested 3198. Please try again in 3.762s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}} Retrying: 1 time(s)...
✨ Generating up to 6 goldens using DeepEval (using gpt-4o and text-

In [ ]:
dataset

EvaluationDataset(test_cases=[], goldens=[Golden(input='How do global Diwali festivities in the UK, US, and Pakistan highlight cultural exchange and impact?', actual_output=None, expected_output="Global Diwali festivities serve as a platform for cultural exchange and highlight the impact of multicultural engagement in several countries. In the UK, Diwali celebrations, such as those at the Swaminarayan Temple in Neasden, are attended by national figures like former Prince Charles, emphasizing the contributions of the Hindu community to British society. The event is marked by lights, decorations, and cultural festivities, fostering mutual appreciation and integration. In the US, Diwali's significance was formally recognized by Congress in 2007, with President Obama participating in White House celebrations in 2009, reflecting recognition and respect for cultural diversity. In Pakistan, the exchange of traditional sweets between Indian and Pakistani border troops during Diwali symbolizes 

In [ ]:
goldens[0]

Golden(input='How do global Diwali festivities in the UK, US, and Pakistan highlight cultural exchange and impact?', actual_output=None, expected_output="Global Diwali festivities serve as a platform for cultural exchange and highlight the impact of multicultural engagement in several countries. In the UK, Diwali celebrations, such as those at the Swaminarayan Temple in Neasden, are attended by national figures like former Prince Charles, emphasizing the contributions of the Hindu community to British society. The event is marked by lights, decorations, and cultural festivities, fostering mutual appreciation and integration. In the US, Diwali's significance was formally recognized by Congress in 2007, with President Obama participating in White House celebrations in 2009, reflecting recognition and respect for cultural diversity. In Pakistan, the exchange of traditional sweets between Indian and Pakistani border troops during Diwali symbolizes harmonious gestures that facilitate camara